<a href="https://colab.research.google.com/github/bharatgirdhar/DeepVisionProject/blob/main/models/Custom_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

#        self.dropout=nn.Dropout(0.05)
# Prep Layer
        
        self.conv0 = nn.Conv2d(3, 64, 3, padding=1, stride=1, bias=False) #32
        self.Norm0=nn.BatchNorm2d(64)

# Layer 1
#X
        self.conv1 = nn.Conv2d(64, 128, 3, padding=1, stride=1, bias=False) #32
        self.pool1 = nn.MaxPool2d(2, 2)
        self.Norm1=nn.BatchNorm2d(128)

#Res Block - R1
        self.conv1_1 = nn.Conv2d(128, 128, 3, padding=1, stride=1, bias=False) #32
        self.Norm1_1=nn.BatchNorm2d(128)

        self.conv1_2 = nn.Conv2d(128, 128, 3, padding=1, stride=1, bias=False) #32
        self.Norm1_2=nn.BatchNorm2d(128)

# Layer 2
        self.conv2 = nn.Conv2d(128,256, 3, padding=1, stride=1, bias=False) #32
        self.pool2 = nn.MaxPool2d(2, 2)
        self.Norm2=nn.BatchNorm2d(256)

# Layer 3
#X
        self.conv3 = nn.Conv2d(256, 512, 3, padding=1, stride=1, bias=False) #32
        self.pool3 = nn.MaxPool2d(2, 2)
        self.Norm3=nn.BatchNorm2d(512)

#Res Block - R2
        self.conv3_1 = nn.Conv2d(512, 512, 3, padding=1, stride=1, bias=False) #32
        self.Norm3_1=nn.BatchNorm2d(512)

        self.conv3_2 = nn.Conv2d(512, 512, 3, padding=1, stride=1, bias=False) #32
        self.Norm3_2=nn.BatchNorm2d(512)

        self.pool4 = nn.MaxPool2d(4, 4)

        self.fc1=nn.Linear(in_features=512, out_features=10)


    def forward(self, x):
        
 # Prep Layer
        x = F.relu(self.Norm0(self.conv0(x)))
        #print(x.shape)
# Layer 1
        x = F.relu(self.Norm1(self.pool1(self.conv1(x))))

# Res Block
        #print(x.shape)
        #R1=x
        R1 = F.relu(self.Norm1_2(self.conv1_2(F.relu(self.Norm1_1(self.conv1_1(x))))))
        #x = self.conv7(x)
        x = torch.add(x, R1)
# Layer 2
        x = F.relu(self.Norm2(self.pool2(self.conv2(x))))
        
# Layer 3
        x = F.relu(self.Norm3(self.pool3(self.conv3(x))))

# Res Block
        #print(x.shape)
        #R2=x
        R2 = F.relu(self.Norm3_2(self.conv3_2(F.relu(self.Norm3_1(self.conv3_1(x))))))
        #x = self.conv7(x)
        x = torch.add(x, R2)
        
        #print(x.shape)
        x = self.pool4(x)
        
        x = x.view(-1, 512)
        #print(x.shape)
        x=self.fc1(x)
        return F.log_softmax(x)